# Libraries

In [1]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import shutil
import pandas as pd
from tqdm import tqdm
import time

In [2]:
notebooks_path = Path.cwd()
repo_path = notebooks_path.parent
print(f'current directory is: {notebooks_path}')

import utils_ric as utils
from info import patient

current directory is: d:\VS_Projects\MIRA_FINAL_PROJECT\MIRA_FINAL_PROJECT\notebooks


# Functions

In [3]:
def image_registration(pat, fixed_image, moving_image, param_files=None, mask=False):
    """Give two images and they will be registered. The outputs are the moving image registered the transformation map.

    Args:
        fixed_image (sitk image): fixed (template) image
        moving_image (sitk image): moving image (image that will be transformed)

    Returns:
        sitk image, transformix map: transformed image and the transformation map
    """
    #Start registration settings
    elastixImageFilter = sitk.ElastixImageFilter() #Image filter object
    #Set parameter file
    if param_files is not None: #if the user gives a parameter file
        if len(param_files) == 1: #if the user gives only one parameter file
            elastixImageFilter.SetParameterMap(param_files[0])
        else: #if the user gives more than one parameter file
            parameterMapVector = sitk.VectorOfParameterMap()
            for param_file in param_files:
                parameterMapVector.append(param_file)
            elastixImageFilter.SetParameterMap(parameterMapVector)
    #Set images
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    #set masks
    if mask:
        fixed_mask = sitk.ReadImage(str(repo_path / 'data' / 'masks' / f'pat{pat.pat_num}_i_lungmask.nrrd'))
        moving_mask = sitk.ReadImage(str(repo_path / 'data' / 'masks' / f'pat{pat.pat_num}_e_lungmask.nrrd'))
        #change spacing to match the images
        fixed_mask.SetSpacing(fixed_image.GetSpacing())
        moving_mask.SetSpacing(moving_image.GetSpacing())
        elastixImageFilter.SetFixedMask(fixed_mask)
        elastixImageFilter.SetMovingMask(moving_mask)
    
    #Run registration
    elastixImageFilter.Execute()

    #Get result image
    resultImage = elastixImageFilter.GetResultImage()

    #Transformation map
    transformParameterMap = elastixImageFilter.GetTransformParameterMap()
    
    return resultImage, transformParameterMap

In [4]:
def read_param_files(name_list):
    """read list of parameter files and convert to list of sitk parameter maps

    Args:
        name_list (list): list with name of the parameter maps

    Returns:
        list: list of sitk parameter maps
    """
    param_files = [] #list of parameter maps
    for path in name_list: #iterate over the list of parameter files
        param_files.append(sitk.ReadParameterFile(str(repo_path / 'data/parameterfiles' / path))) #transform to sitk parameter map and append to list
    return param_files

In [5]:
def register_points(pat, transformParameterMap, points_path, transformation_name):
    """Register points using the transformation map
        IMPORTANT: the points must be in the same space as the moving image
            - This happens because the transformation maps the moving image to the fixed image
            - This is contrary to what usually happens in image registration
            - See SimpleElastix documentation for more information

    Args:
        transformParameterMap (transformix map): transformation map
        points_path (str): path to pts file (txt file with points)
        moving_image (sitk image): moving image (reference image)
        pat (obj): patient object

    Returns:
        Nothing: the points are saved in a txt file
    """
    #Transformix filter object
    transformixImageFilter = sitk.TransformixImageFilter()
    #Set transformation map
    transformixImageFilter.SetTransformParameterMap(transformParameterMap)
    #Set points
    transformixImageFilter.SetFixedPointSetFileName(str(repo_path / points_path))
    #Set moving image (needed to get spacial information)
    transformixImageFilter.SetMovingImage(pat.im_sitk('e')) 
    #Run transformation
    transformixImageFilter.Execute()
    
    #change name and relocate outputpoints.txt file
    old_name = notebooks_path / 'outputpoints.txt'
    new_name = repo_path / 'data/transformed_keypoints' / f'outputpoints_pat{pat.pat_num}_trans-{transformation_name}.txt'
    shutil.move(old_name, new_name)

In [6]:
def read_outputpoints(points_name):
    """Read the outputpoints.txt file and return the coordinates of the points
    
        args:
            points_name (str): path to the outputpoints.txt file
        returns:
            numpy array: array with the coordinates of the points
    """ 
    df = pd.read_csv(str(repo_path / f'data/transformed_keypoints' / points_name), sep="\t", header=None)
    #get column 5 (where the output points are)
    outpoints = df.iloc[:, 5]
    #remove text to keep just x y z coordinates
    outpoints = outpoints.str.replace('; OutputPoint = \[ ', '', regex=True)
    outpoints = outpoints.str.replace(' \]', '', regex=True)
    #transform to numpy the series with the coordinates separated by \tab
    outpoints = outpoints.str.split(' ', expand=True).to_numpy(dtype=float)
    return abs(outpoints) #absolute value because the z coordinates are negative (we inverted the axis for visualization)

In [7]:
def compute_tre(pat, transformation_name, start_time, show=False, save=False):
    """compute the mean and std of the tre between the exhale points and the transformed points

    Args:
        pat (obj): patient object
        transformation_name (str): transformation name, used to store the csv file
        show (bool, optional): print mean and std. Defaults to False.
        save (bool, optional): save csv file. Defaults to False.
    """
    #read txt file with transformed points
    points_name = f'outputpoints_pat{pat.pat_num}_trans-{transformation_name}.txt'
    outpoints = read_outputpoints(points_name)
    #get exhale points to compare
    exhale_points = pat.get_landmark('e')
    #compute tre
    tre = utils.calculate_tre(outpoints, exhale_points)
    if show:
        #print mean and std
        print(f'mean tre: {np.mean(tre)}')
        print(f'std tre: {np.std(tre)}')

    #save as csv file
    if not save: #if save is False, exit function
        return
    #defined csv file name
    csv_name = f'tre_trans-{transformation_name}.csv'
    csv_path = repo_path / 'data/results' / csv_name
    #check if file already exists
    if csv_path.is_file():
        #if it exists, read it
        df = pd.read_csv(csv_path)
        if len(df)==4: #if the length is 4, it means that the csv file is full, so we exit the function
            return
        #add new row
        df = pd.concat([df, 
            pd.DataFrame([{
                'patient': pat.pat_num,
                'mean_tre': np.mean(tre),
                'std_tre' : np.std(tre),
                'time': time.time() - start_time
            }])], ignore_index=True
        )
    else: #if it doesn't exist, create it
        df = pd.DataFrame(
            {
                'patient': pat.pat_num,
                'mean_tre': np.mean(tre),
                'std_tre' : np.std(tre),
                'time': time.time() - start_time
            }, index=[0]
        )
    #save csv file
    df.to_csv(csv_path, index=False)

In [8]:
def get_param_files(param_names, mask=False):
    """read list of param file names and return sitk parameter maps and transformation name

    Args:
        param_names (list): list of param file names

    Returns:
        list, str: list of sitk parameter maps, transformation name to save the outputs
    """
    mask_status = 'mask' if mask else 'no_mask'
    param_files = read_param_files(param_names) #read files to sitk objects
    transformation_name = Path(param_names[-1]).stem + f'_comp-{len(param_names)}_{mask_status}' #name to save the outputs
    
    return param_files, transformation_name


# MAIN

## Image registration

Now we can register the images. We just have to give the fixed image, moving image and the parameter file.

In [11]:
#INPUT USER
param_names = ['elastix_default_bspline_common_nres5_mni1000.txt', 'elastix_default_affine_common_nres5_mni1000.txt'] #list of files to use
mask = True #use mask or not

for i in tqdm(range(1, 5)): #go through all patients
    #start timer
    start_time = time.time()
    #define patient
    pat = patient(num=i)
    #Get param map files and transformation name
    param_files, transformation_name = get_param_files(param_names, mask) #read files to sitk objects and get transformation name
    #run registration
    _, transformParameterMap = image_registration(pat, fixed_image=pat.im_sitk('i'), moving_image = pat.im_sitk('e'), param_files=param_files, mask=mask)
    #transform the points. They will be saved in a txt file
    register_points(pat, transformParameterMap, points_path = pat.points_path(type='i', format='pts'), transformation_name=transformation_name)
    #compute tre and save csv file
    compute_tre(pat, transformation_name, start_time, show=True, save=True)

 25%|██▌       | 1/4 [06:19<18:58, 379.64s/it]

mean tre: 1.8143107356864778
std tre: 2.3454930062075774


 50%|█████     | 2/4 [11:27<11:14, 337.13s/it]

mean tre: 4.3622983305643155
std tre: 5.414906142764656


 75%|███████▌  | 3/4 [17:48<05:57, 357.58s/it]

mean tre: 1.4814670796092793
std tre: 1.2236635024392597


100%|██████████| 4/4 [23:12<00:00, 348.17s/it]

mean tre: 3.277506131946995
std tre: 3.6612369579834043
